In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import os
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

data_dir = './Inbreast_Dataset'

img_width, img_height = 128, 128
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_generator, epochs=5, validation_data=validation_generator)

score = model.evaluate(validation_generator, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

2024-04-10 11:45:40.662406: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 11:45:40.664223: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 11:45:40.687819: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 11:45:40.687842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 11:45:40.688696: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Found 87 images belonging to 2 classes.
Found 21 images belonging to 2 classes.
Epoch 1/5
3/3 [==============================] - 9s 3s/step - loss: 0.7107 - accuracy: 0.5747 - val_loss: 0.7094 - val_accuracy: 0.6667
Epoch 2/5
3/3 [==============================] - 7s 3s/step - loss: 0.6626 - accuracy: 0.5977 - val_loss: 0.6679 - val_accuracy: 0.6667
Epoch 3/5
3/3 [==============================] - 7s 2s/step - loss: 0.6425 - accuracy: 0.6552 - val_loss: 0.6378 - val_accuracy: 0.6667
Epoch 4/5
3/3 [==============================] - 6s 2s/step - loss: 0.6010 - accuracy: 0.6782 - val_loss: 0.6208 - val_accuracy: 0.6667
Epoch 5/5
3/3 [==============================] - 7s 2s/step - loss: 0.5692 - accuracy: 0.7586 - val_loss: 0.6297 - val_accuracy: 0.7143
Validation loss: 0.6296679973602295
Validation accuracy: 0.7142857313156128


In [2]:
y_pred = model.predict(validation_generator)
y_pred = (y_pred > 0.5).astype(int)

true_labels = validation_generator.classes

print(classification_report(true_labels, y_pred))

1/1 [==============================] - 1s 1s/step
              precision    recall  f1-score   support

           0       0.33      0.14      0.20         7
           1       0.67      0.86      0.75        14

    accuracy                           0.62        21
   macro avg       0.50      0.50      0.47        21
weighted avg       0.56      0.62      0.57        21



In [3]:
data_dir = './Inbreast_Dataset'

img_width, img_height = 128, 128
batch_size = 32

datagen = ImageDataGenerator(rescale=1. / 255)

generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(128, activation='relu')(x)

predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

features = model.predict(generator, verbose=1)

labels = generator.classes

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.3, random_state = 13)

Found 108 images belonging to 2 classes.
4/4 [==============================] - 7s 2s/step


In [4]:
svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[ 0 12]
 [ 0 21]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.64      1.00      0.78        21

    accuracy                           0.64        33
   macro avg       0.32      0.50      0.39        33
weighted avg       0.40      0.64      0.49        33



/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[ 2 10]
 [ 8 13]]

Classification Report:
              precision    recall  f1-score   support

           0       0.20      0.17      0.18        12
           1       0.57      0.62      0.59        21

    accuracy                           0.45        33
   macro avg       0.38      0.39      0.39        33
weighted avg       0.43      0.45      0.44        33

